<img src="https://lh3.googleusercontent.com/pw/ACtC-3fFHZrzKpHGWl0vYz7Sr8FX8QqLQ_tc8XHBSwqQnM4hgsIOjtjaOde1M9oHSAfe1Fs2SwVORlapit4-JOz0mjP8Tnz6HetkLZDZb8CifSd0uoSp1Nj3wG_wh1sEQlKXXzvEA9Y9HnQqu2Ecv2igmInb=w1097-h235-no?authuser=0" alt="2020年度ゲノム情報解析入門" height="100px" align="middle">

# Pythonの基礎 (2)


## はじめに

　今回は、Pythonの基礎（後半）として、データ処理の基礎（テキストファイルを読み込み、任意データの取得、数値計算などの方法）を学びます。テキストの最後には、取得したデータをグラフにしてみます。

### サンプルデータの概要

　次世代シーケンサーから得られるデータは、多量の短いDNA配列です。その短いDNA配列は**リード (Read)**と呼ばれています。

　リードを解析する手法は研究目的によって異なりますが、モデル生物のようにゲノム配列を利用できる場合には、そのゲノム配列にリードを並べる（**アライメントする (Alignment)**）ことから始めます。なお、リードを並べるために使われるゲノム配列のことを、**リファレンスや参照配列 (Reference)**と言います。

　リードのアライメントの概要は、次の2ステップです。
1. リファンレンス中からリードのDNA配列とよく似たDNA配列の場所を探し、
1. その場所にリードを並べます。

下の図は、リードをリファレンスにアライメントしたとき様子を表しています。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3fN9G6os7hlCG31Hj5GWg12CCHWAFApM3gzhD8dOPzD_G3ODDVEdqJTZ7ErILGK3FOl7W0cOjATK9ismYBWh3mesMdGnSFCVzpJKzBFGyE8XHfpJhaI4Dx-wKV8ihiuZ-ekL5B-Xy9SA2r4OzBTReWh=w1693-h924-no?authuser=0" alt="read_alingment" height="300px">

　次世代シーケンサーから得られるリードは多量にあるため、リファレンス上には多数のリードが並べられます。ここで、リファレンスのある塩基座位に注目すると、その座位に0本以上のリードが置かれているはずです。そのリードの本数のことを **リードデプス (Read depth)** と言います。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3fc0NCy3Rz34WM-3x2wiYi3v5eHRfUFPC8tdHcDrsozKoeRKULKfFCTMLU7gbgqKZ92KmT3cg1KV-SopJFqyAn92mJZh1byhjf0mHdhg_8O6DpOr3eO6_-aA8nRchjlCOSJaOibdeLjgkHpL0fO_UZV=w1649-h1258-no?authuser=0" alt="read_depth" height="400px">

　また、ときには、リファレンスとリード上の塩基が異なる場合もあるでしょう。このようなサンプル間（ここでは、リファレンスとシーケンスサンプル間）で異なる塩基のことを **SNP (Single Nucleotide Polymorphism**)と言います。

　今回の実習で使用する[サンプルデータ](https://raw.githubusercontent.com/CropEvol/lecture/master/data/allele_count_mini.txt)は、リファレンスにリードをアライメントしたときの状況を表したデータです。

　データはテキストファイルに書かれています。各行には1塩基座位分の情報が書かれており、左から下記の情報が書かれています。
- 染色体名
- 塩基座位番号
- リファレンスの塩基
- リード上のリファレンスの塩基とは異なる塩基  
  <small>異なる塩基が見つからなかった場合は「.」が書かれている</small>
- リファレンスと同じ塩基をもつリード数
- リファレンスと異なる塩基をもつリード数

<img src="https://lh3.googleusercontent.com/pw/ACtC-3fw6vfK8BRwFBvBX_ql2kioDfSxn9j1tAT1WiTsvDMCoHF1gVb7cssdzhqXW231IXrbUH9PsefMS8fdnZucj6ugeZecTQBjF06fQs9pBlbIclaPGtqhLoNq7QvfyAEtSjJ1xECMSksqskGkMO5x2nnu=w906-h727-no?authuser=0" alt="sample_textfile" height="250px">


### テキストファイル処理の基本

　テキストファイル処理の基本は、繰り返し処理を使って、「**テキストファイルを上の行から順番に1行ずつ処理する**」です。その基本を念頭に、今回の実習に取り組んでください。

### サンプルデータのダウンロード


　実習前に、下記のコードセルを実行して、[サンプルデータ](https://raw.githubusercontent.com/CropEvol/lecture/master/data/allele_count_mini.txt)をダウンロードしましょう。サンプルデータは、「allele_count_mini.txt」というファイル名でGoogle Colab上にダウンロードされます。

　ついでに、[実習用のサンプルデータ](https://raw.githubusercontent.com/CropEvol/lecture/master/data/allele_count_mini2.txt)「allele_count_mini2.txt」もダウンロードします。

In [ ]:
"""
＊重要＊
最初にこのセルを実行してください。
この実習で使うサンプルファイルをダウンロードします。
"""
# 例題用サンプルデータ 
!wget https://raw.githubusercontent.com/CropEvol/lecture/master/data/allele_count_mini.txt  -O allele_count_mini.txt
# 実習用サンプルデータ 
!wget https://raw.githubusercontent.com/CropEvol/lecture/master/data/allele_count_mini2.txt -O allele_count_mini2.txt

## 今回の実習内容
1. ファイルを開く・閉じる
1. 1行ずつ読み込む
1. 1行の中身（要素）を分割する
1. 数値計算をおこなう
1. 数値計算結果をリストに保存する
1. グラフを描いてみる

---

## 1. ファイルを開く・閉じる

　テキストファイルの中のデータを処理するためには、まずテキストファイルを開く必要があります。ここでは、テキストファイルを開いて、閉じる操作を学びます。

　テキストファイルを開くためには、`open`関数を使います。また、閉じるときには`close`関数を使います。

```python
ファイル変数 = open("<テキストファイルの名前>")
ファイル変数.close()
```

<small>※ `ファイル変数.close()`は、正確に言うと「関数」ではありません。「メソッド」と呼ばれるものです。この違いをきちんと説明するには、「オブジェクト志向」という少々込み入った話をしなければならなくなるため、「ゲノム情報解析入門」では、メソッドも関数と表現して説明しています。関数とメソッドの違いは、Pythonに慣れてきてから自分で勉強してみてください。</small>

In [ ]:
# 例
f = open("allele_count_mini.txt")  # ファイルを開く
print(f)
f.close()  # ファイルを閉じる

### 実習1

　`open`関数を使って、実習用サンプルデータファイル`allele_count_mini2.txt`を開くコードを追記してください。



In [ ]:
f = 
print(f)
f.close()

#### 解答例

In [ ]:
f = open("allele_count_mini2.txt")
print(f)
f.close()

## 2. 1行ずつ読み込む

　次は、テキストファイルの中身を1行ずつ表示してみましょう。前回習った`for`構文を使います。

```python
ファイル変数 = open("<テキストファイルの名前>")
for 変数 in ファイル変数:
  print(変数)
ファイル変数.close()
```
　各ループで、1行分のデータが取り出されます。ここでは取り出された1行分のデータを単に表示してみます。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3eC1SxhZADaS8yB-keUjWzAaUCrFAWv17eMvzqtQbAltWZB1TEbL72fCgyTIvxJbzTyU3JkAjlhuiV46fsSBOo-bgCB8t8CYbdUdAGzhZ_LWKsb5LjD-S7Zdw2dPen_lG_el25GgdF9AUZGjhkxewRT=w2880-h774-no?authuser=0" alt="read_line" height="250px">

In [ ]:
# 例
f = open("allele_count_mini.txt")

# 各ループで、ファイルの中身が1行ずつ取り出されて、変数lineに入る
for line in f:
  print(line) # 表示
f.close()

　行と行のあいだに、不要な空白行が挿入されてしまっています。これは、元々各行に含まれている**改行文字 (`\n`)**に加え、`print`関数により新たに改行文字が追加されてしまうためです。

　各行に含まれている改行文字は、この後の「3. 1行の中身（要素）を分割する」で処理します。

<small>プログラミングをしていると、改行文字のような特殊文字に遭遇します。特殊文字は、テキストファイル中ではただ単に見えないというだけで、ファイルに書き込まれています。</small>

<img src="https://lh3.googleusercontent.com/pw/ACtC-3cxD7-JBmbAhjxPkLq-lxDBI_ukwkllQ1VJjKYW_cb1HCxekOkW-f7axW_juukwGmrAKRGNzu104gogJQEXuD6qow2YcTRo_BiX02xWuAXd8054fWY3RNJ7B577UGIiz2U3oSSpo8etmbea19D_1wuM=w931-h783-no?authuser=0" alt="newline_code" height="300px">

### 実習2

　for構文を使って、テキストファイルの中身を1行ずつ表示してください。

In [ ]:
f = open("allele_count_mini2.txt")

f.close()

#### 解答例

In [ ]:
f = open("allele_count_mini2.txt")
for line in f:
  print(line)
f.close()

## 3. 1行の中身（要素）を分割する

　今回のサンプルデータには、1行に複数の情報が入っています（染色体名、塩基座位番号、など）。それぞれの情報にアクセスするには、1行のデータをうまく分割して、アクセス可能な形に変換する必要があります。

　1行のデータを分割するために、文字列データを分割する便利な関数 `split()` を使います。分割後のデータは、**リスト (list)**と呼ばれる型のデータに変換されます。なお、リスト変換すると、改行文字は都合よく取り除かれます。

```python
ファイル変数 = open("<テキストファイルの名前>")
for 変数 in ファイル変数:
  変数 = 変数.split()  # 1行データをリスト状にする
  print(変数)
ファイル変数.close()
```

<img src="https://lh3.googleusercontent.com/pw/ACtC-3e9dx-8eoin_R8YgK0m-khjRAW0Xw05H-jJ-UbfRWNOvzn1-h8Y-94_PRNbTpfDvCqNmnw3kQLew1-64fY7dRkFmcffTuSQqdFlc_puqOo04J5hLYA80g5x2pmWfof_oBBE6X4GynvXyeA1tttPHI1w=w1795-h227-no?authuser=0" alt="line_to_list" height="80px">

In [ ]:
# 例
f = open("allele_count_mini.txt")

for line in f:
  line = line.split()  # 1行データをリストにする
  print(line)
f.close()

### 実習3

　1行分のデータを分割して、リストデータにしてください。

In [ ]:
f = open("allele_count_mini2.txt")
for line in f:
  # 1行データをリストにする
  line = 
  print(line)
f.close()

#### 解答例

In [ ]:
f = open("allele_count_mini2.txt")
for line in f:
  line = line.split()
  print(line)
f.close()

## 4. 数値計算をおこなう

### 4-1. 各要素の取り出し方

　リスト上の各情報のこと **要素 (item)** と言います。リスト化されたデータから、必要な要素を取り出す方法を学びましょう。

　文字列と同じように、各要素にも位置番号（**インデックス**）が振られています。任意の要素にアクセスするには、そのインデックスを指定します。

```python
ファイル変数 = open("<テキストファイルの名前>")
for 変数 in ファイル変数:
  変数 = 変数.split()
  print(変数[インデックス])  # 要素を取り出して表示する
ファイル変数.close()
```

<img src="https://lh3.googleusercontent.com/pw/ACtC-3eMsGNy6qgXSfjtkLBaTismAP_IRrcTgUx-bJXalWUlZGvxOfp4bXj2xkmanUKfAtLDHQKBRIGZxs1ZXXsgpWx4oZk0KUJ3lanSYGiDk_WNBJdbGUq4VTaIR5cvwfmCgUqwMHYdK3vFp7JRO3aTYsXU=w1674-h393-no?authuser=0" alt="list_index" height="150px">

In [ ]:
# 例
f = open("allele_count_mini.txt")

for line in f:
  line = line.split()
  print(line, line[0]) # 要素を取り出して表示する
f.close()

### 実習4-1

　分割した1行分のデータ（リストデータ）から、4番目と5番目の要素を取り出してください。

In [ ]:
f = open("allele_count_mini2.txt")
for line in f:
  line = line.split()
  print(line)
f.close()

#### 解答例

In [ ]:
f = open("allele_count_mini2.txt")
for line in f:
  line = line.split()
  print(line, line[4], line[5])
f.close()

### 4-2. 数値計算

　次に、要素の情報を使って、数値計算してみましょう。数値計算は、下記のような簡単な記述でおこなえます。

```python
# 足し算: 半角のプラス記号を使う
数値 + 数値
# 引き算: 半角のマイナス記号を使う
数値 - 数値
# 掛け算: 半角のアスタリスク記号を使う
数値 * 数値
# 割り算: 半角のスラッシュ記号を使う
数値 / 数値
```

 <small>これ以外の数値計算（例えば、「べき乗」計算など）ももちろん可能です。その方法は自分で調べてみてください（検索ワード `python　べき乗`）。</small>


In [ ]:
# 数値計算例
print(2 + 3)
print(2 - 3)
print(2 * 3)
print(2 / 3)

　サンプルデータに戻り、1行分のデータを再確認しておきます。
- 0番目の要素: 染色体名
- 1番目の要素: 塩基座位番号
- 2番目の要素: リファレンスの塩基
- 3番目の要素: リード上のリファレンスの塩基とは異なる塩基  
- 4番目の要素: リファレンスと同じ塩基をもつリード数
- 5番目の要素: リファレンスと異なる塩基をもつリード数

　4番目と5番目の数値を足し合わせると、各塩基座位上の総リード数（リードデプス）になります。ここでは、各塩基座位のリードデプスを算出して表示してみましょう。


In [ ]:
# 失敗例
f = open("allele_count_mini.txt")

for line in f:
  line = line.split()
  c1 = line[4]      # リファレンスと同じ塩基のリード数
  c2 = line[5]      # リファレンスと異なる塩基のリード数
  d = c1+c2   # リードデプスを求める
  print(line, d)
f.close()

　上の失敗例を実行するとわかるように、数値計算されていません。これは、数字が「文字列」として扱われており、前回のテキストで習った「文字列の結合」がおこなわれてしまっているからです。

　数字を「文字列」から「数値」に変換して、数値計算できるようにしましょう。数値変換をおこなうには、`int()`という関数を使います。

```python
# 文字列 => 数値の変換
int("<数字>")
```

In [ ]:
# 文字列 => 数値の変換
print(int("2") + int("3"))
# 数値 => 文字列の変換
print(str(2) + str(3))

In [ ]:
# 成功例
f = open("allele_count_mini.txt")

for line in f:
  line = line.split()
  c1 = int(line[4])  # リファレンスと同じ塩基のリード数; 数値変換
  c2 = int(line[5])  # リファレンスと異なる塩基のリード数; 数値変換
  d = c1 + c2 # リードデプスを求める
  print(line, d)
f.close()

### 実習4-2

　各塩基座位について、変数`c1`（リファレンスと同じ塩基をもつリード数）と`c2`（リファレンスと異なる塩基をもつリード数）の値を足し算して、総リード数（リードデプス）を出力してください。

In [ ]:
f = open("allele_count_mini2.txt")
for line in f:
  line = line.split()
  c1 = line[4]       # リファレンスと同じ塩基のリード数
  c2 = line[5]       # リファレンスと異なる塩基のリード数
  # リードデプスを求める
  d = 
  print(line, d)
f.close()

#### 解答例

In [ ]:
f = open("allele_count_mini2.txt")
for line in f:
  line = line.split()
  c1 = int(line[4])       # リファレンスと同じ塩基のリード数; 数値変換
  c2 = int(line[5])       # リファレンスと異なる塩基のリード数; 数値変換
  # リードデプスを求める
  d = c1 + c2
  print(line, d)
f.close()

## 5. 数値計算結果をリストに保存する

　このあと、リードデプスの推移のグラフを描画するために、各塩基座位のリードデプスを新しいリストに格納してみましょう。

　リスト作成には角カッコ`[]`を使います。また、既存のリストに値を追加するには`append()`という関数を使います。

```python
# 新しい空のリストの作成
リスト変数 = []
# 既存のリストに値を追加
リスト変数.append(値)
```

　今回の場合、リストの作成は`for`構文の前におこないます。リストへの追加処理は、`for`構文中の1塩基座位のリードデプス情報を得たあと（`d=c1+c2`のあと）におこないます。

In [ ]:
# 例
f = open("allele_count_mini.txt")

depth = []  # 各リードデプス用の空のリスト

for line in f:
  line = line.split()
  c1 = int(line[4])
  c2 = int(line[5])
  d = c1 + c2
  depth.append(d) # リードデプス情報をリストに追加
  print(depth)     # リスト表示
f.close()

### 実習5

　リスト変数`depth`に、各塩基座位のリードデプス情報を追加してください。

In [ ]:
f = open("allele_count_mini2.txt")

depth = [] # 各リードデプス用の空のリスト

for line in f:
  line = line.split()
  c1 = int(line[4])
  c2 = int(line[5])
  d = c1 + c2
  #リードデプスをリストに入れる

  print(depth)     # リスト表示
f.close()

#### 解答例

In [ ]:
f = open("allele_count_mini2.txt")

depth = []

for line in f:
  line = line.split()
  c1 = int(line[4])
  c2 = int(line[5])
  d = c1 + c2
  #リードデプスをリストに入れる
  depth.append(d)
  print(depth)     # リスト表示
f.close()

## 6. グラフを描いてみる

　最後に、「5. 数値計算結果をリストに保存する」で得たリードデプスの情報を使って、リードデプスの推移をグラフにしてみましょう。

<small>※ ここでは、下記のプログラムの解説はおこないません。実行をおこない、グラフ描画ができることを確かめるだけです。グラフ描画については、後々の実習で勉強します。</small>

In [ ]:
# 例
f = open("allele_count_mini.txt")

depth = []  # 各リードデプス用の空のリスト

for line in f:
  line = line.split()
  p = line[1]
  c1 = int(line[4])
  c2 = int(line[5])
  d = c1 + c2
  depth.append(d)  # リードデプスをリストに追加
f.close()

# グラフ描画
# （各塩基座位のリードデプスを折れ線グラフにする）
import matplotlib.pyplot as plt
plt.plot(depth)
plt.xlabel("position")
plt.ylabel("read depth")
plt.show()

### 実習6

　`plt.plot(depth, color="blue")`のグラフの色を指定している部分`color="blue"`を、`color="orange"`に変更してみましょう。

In [ ]:
f = open("allele_count_mini2.txt")

depth = []  # 各リードデプス用の空のリスト

for line in f:
  line = line.split()
  p  = line[1]
  c1 = int(line[4])
  c2 = int(line[5])
  d  = c1 + c2
  depth.append(d)  # リードデプスをリストに追加
f.close()

# グラフ描画
# （各塩基座位のリードデプスを棒グラフにする）
import matplotlib.pyplot as plt
plt.plot(depth, color="blue")
plt.xlabel("position")
plt.ylabel("read depth")
plt.show()

### 解答例

In [ ]:
f = open("allele_count_mini2.txt")

depth = []  # 各リードデプス用の空のリスト

for line in f:
  line = line.split()
  p  = line[1]
  c1 = int(line[4])
  c2 = int(line[5])
  d  = c1 + c2
  depth.append(d)  # リードデプスをリストに追加
f.close()

# グラフ描画
# （各塩基座位のリードデプスを棒グラフにする）
import matplotlib.pyplot as plt
plt.plot(depth, color="orange")
plt.xlabel("position")
plt.ylabel("read depth")
plt.show()

---
## まとめ

　今回、Pythonの基礎（後半）を勉強しました。
- テキストファイルを読み込み方法
- 文字列の分割方法（リスト化方法）
- リストから要素を取り出す方法
- 数値計算をおこなう方法
- 新しいリストを作成し、そのリストにデータを追加する方法

また、得られたデータをグラフ描画できることも勉強しました。

　今回おこなったように **テキストファイルを上の行から順番に1行ずつ処理する** 方法は、ファイル処理でよくおこなわれている方法です（つまり、テキストファイル処理の基礎です）。この方法は、かなり汎用性が高いです。しかし、今回の場合、2列分のデータを計算したいだけなのに、`for`構文を使い・1行の情報をリストデータに変換し・数字を数値として変換するなど、少々手続きが多く煩雑です。

　Pythonには、テーブル状のデータをもっと容易に処理する**ライブラリ (Library)**と呼ばれる追加機能のようなものがあります。次回からライブラリを使って、もっと快適にデータ解析する方法を学んでいきます。

<small>※ じつは、このテキストでもライブラリを使っていました。説明はしませんでしたが、「6. グラフを描いてみる」のグラフ描画部分です。`matplotlib`というライブラリを使いました。このライブラリについても、次回のテキストで勉強します。</small>
